In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt

In [39]:
def import_dataset(split_percent = 70):
    # dataset = pd.read_csv('E:\ELL_Project\problem2\weather_data.csv')     
    dataset = pd.read_excel('E:\ELL_Project\problem2\weather_data.xlsx')                     
    dataset = dataset.sample(frac = 1)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1:].values
    datasize = X.shape[0]

    split_point = split_percent//10

    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]

    return(X_train,X_test,y_train,y_test)


In [40]:
def feature_scaling(X_train):
    training_size = X_train.shape[0]
    X_mean = np.sum(X_train,axis=0) / training_size
    X_var = np.sqrt(np.sum((np.square(X_train-X_mean)),axis=0)/training_size)
    X_train_reg = (X_train - X_mean) / X_var

    return (X_mean,X_var,X_train_reg)


In [212]:
def KNN(X_train,y_train,X_test,y_test,k_num,feat_scal=True):

    test_size = X_test.shape[0]
    train_size = X_train.shape[0]
    # X_m,X_v,X_r,y_m,y_v,y_r = feature_scaling(X_train,y_train)
    if feat_scal:
        X_m,X_v,X_r = feature_scaling(X_train)
        X_test_reg = (X_test-X_m)/X_v
    else:
        X_r = X_train
        X_test_reg = X_test
    # y_test_reg = (y_test-y_m)/y_v
    y_pred = np.array([])
    # print(y_pred)
    for i in range(test_size):
        sample_X = X_test_reg[i:i+1,:]
        sample_y = y_test[i,:]
        dist_vec = np.sqrt(np.sum(np.square(X_r-sample_X),axis=1))
        dist_vec = dist_vec.reshape((train_size,1))

        d = np.column_stack((dist_vec[:,:],y_train[:,0]))
        # print(d)

        sorted_dist = d[np.argsort(d[:, 0])]
        temp_distance_vector = sorted_dist[:k_num,0]
        distance_vector = temp_distance_vector.reshape(-1,1)
        # distance_vector = np.append(np.ones(distance_vector.shape),distance_vector)
        # print(distance_vector.shape)
        for j in np.arange(0,10,0.5):
            distance_vector = np.append(distance_vector,(temp_distance_vector**j).reshape(-1,1),axis=1)
        distance_vector = distance_vector[:,1:]
        # print(distance_vector.shape)
        ddd = np.sum(distance_vector,axis=0).reshape(1,-1)
        # print(ddd[0,0])
        # print(ddd.shape)

        y_val_vector = sorted_dist[:k_num,1].reshape(-1,1)
        # print(y_val_vector.shape)
        # print(distance_vector.shape)
        # print(ddd.shape)
        num1 = np.dot(np.transpose(y_val_vector),distance_vector) / ddd
        # print(num1.shape)
        # num1 = np.max(num1)
        if i==0:
            # print('check')
            y_pred = num1.reshape(1,-1)
        else:
            # print(y_pred.shape)
            y_pred = np.append(y_pred,num1.reshape(1,-1),axis=0)
            

    # print(y_pred.shape)
    return y_pred
    
        



In [217]:
def accuracy(y_pred,y_test):

    # y_pred = y_pred.reshape(-1,1)
    print('MSE LOSS')
    mse_loss = np.sum(np.square((y_test-y_pred)),axis=0) / y_test.shape[0]
    print(mse_loss)
    print('............................................................................')
    print('MAE LOSS')
    mae_loss = np.sum(np.abs((y_test-y_pred)),axis=0) / y_test.shape[0]
    print(mae_loss)
    # print('............................................................................')

In [223]:
X_train,X_test,y_train,y_test = import_dataset()
y_pred = KNN(X_train,y_train,X_test,y_test,50,False)
accuracy(y_pred,y_test)

MSE LOSS
[2.00623067 2.19407759 2.37097801 2.53666417 2.6914236  2.83582265
 2.97055692 3.0963647  3.21397723 3.32409174 3.42735847 3.52437586
 3.61569033 3.70179855 3.78315069 3.8601543  3.93317804 4.00255543
 4.06858822 4.13154954]
............................................................................
MAE LOSS
[0.97126667 1.01689978 1.05797179 1.09507075 1.12835438 1.15849231
 1.18582385 1.21083836 1.2335726  1.25440691 1.27371248 1.29149648
 1.30803281 1.32373422 1.33835682 1.35220475 1.36534539 1.37771173
 1.3895756  1.40123372]
